## Imports

In [8]:
from helpers import *

# Models
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# Scalers
from sklearn.preprocessing import MinMaxScaler

In [2]:
# feature extraction parameters
opt = {
        'input_folder': r"<IMAGES FOLDER IMAGES ARE STORED>",
        'output_folder': r"<FOLDER WHERE FEATURES ARE STORED",
        'sharpness_factor': 10,
        'bordersize': 3,
        'show_images': False,
        'is_binary': False,
        'LBP_numPoints': 8,
        'LBP_radius':1,
        'LBP_method': 'uniform',
        'HOG_width': 64,
        'HOG_height': 128,
    }

In [3]:
X, Y = read_data("data_set", windows=False)

0start ..
1start ..


In [4]:
data_train, data_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
HOG_feature_training, HINGE_feature_training, COLD_feature_training = extract_features(imgs=data_train, options=opt)

/home/passantabdelgalil/Documents/gender-classification/cold_feature_extraction.py:117: RuntimeWarning: divide by zero encountered in log10
  rhos_log_space = np.log10(rhos)


In [6]:
HOG_feature_testing, HINGE_feature_testing, COLD_feature_testing = extract_features(imgs=data_test, options=opt)

/home/passantabdelgalil/Documents/gender-classification/cold_feature_extraction.py:117: RuntimeWarning: divide by zero encountered in log10
  rhos_log_space = np.log10(rhos)


## Scaling Features

In [ ]:
scaler = MinMaxScaler()
training_features = scaler.fit_transform(training_HOG)
testing_features = scaler.transform(testing_HOG)

## SVM

### HOG

In [10]:
svm_hog = LinearSVC(C=100, random_state=42, max_iter=2000)
svm_hog = svm_hog.fit(HOG_feature_training, y_train)
test_pred_hog_svm = svm_hog.predict(HOG_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_hog_svm == y_test)/len(y_test) * 100))

Testing accuracy is 73.97


### HINGE

In [22]:
svm_hinge = LinearSVC(C=100, random_state=42, max_iter=5000)
svm_hinge = svm_hinge.fit(HINGE_feature_training, y_train)
test_pred_hinge_svm = svm_hinge.predict(HINGE_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_cold_svm == y_test)/len(y_test) * 100))

Testing accuracy is 69.86


### COLD

In [43]:
svm_cold = LinearSVC(C=80, random_state=42, max_iter=5000)
svm_cold = svm_cold.fit(COLD_feature_training, y_train)
test_pred_cold_svm = svm_cold.predict(COLD_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_cold_svm == y_test)/len(y_test) * 100))

Testing accuracy is 72.60


## Gradient Boosting Classifier

### HOG

In [24]:
gradient_booster_hog = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
gradient_booster_hog = gradient_booster_hog.fit(HOG_feature_training, y_train)
test_pred_hog_gradient = gradient_booster_hog.predict(HOG_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_hog_gradient == y_test)/len(y_test) * 100))

Testing accuracy is 72.60


### HINGE

In [25]:
gradient_booster_hinge = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
gradient_booster_hinge = gradient_booster_hinge.fit(HINGE_feature_training, y_train)
test_pred_hinge_gradient = gradient_booster_hinge.predict(HINGE_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_hinge_gradient == y_test)/len(y_test) * 100))

Testing accuracy is 82.19


### COLD

In [26]:
gradient_booster_cold = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
gradient_booster_cold = gradient_booster_cold.fit(COLD_feature_training, y_train)
test_pred_cold_gradient = gradient_booster_cold.predict(COLD_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_cold_gradient == y_test)/len(y_test) * 100))

Testing accuracy is 57.53


## LogisticRegression

### HOG

In [27]:
lg_hog = LogisticRegression(random_state=42, max_iter=1000, C=10, class_weight="balanced")
lg_hog = lg_hog.fit(HOG_feature_training, y_train)
test_pred_hog_lg = lg_hog.predict(HOG_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_hog_lg == y_test)/len(y_test) * 100))

Testing accuracy is 75.34


### HINGE

In [28]:
lg_hinge = LogisticRegression(random_state=42, max_iter=1000, C=10, class_weight="balanced")
lg_hinge = lg_hinge.fit(HINGE_feature_training, y_train)
test_pred_hinge_lg = lg_hinge.predict(HINGE_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_hinge_lg == y_test)/len(y_test) * 100))

Testing accuracy is 63.01


### COLD

In [29]:
lg_cold = LogisticRegression(random_state=42, max_iter=1000, C=10, class_weight="balanced")
lg_cold = lg_cold.fit(COLD_feature_training, y_train)
test_pred_cold_lg = lg_cold.predict(COLD_feature_testing)
print(f"Testing accuracy is %0.2f"%(np.sum(test_pred_cold_lg == y_test)/len(y_test) * 100))

Testing accuracy is 63.01


# Voting Classifier

- Logistic Regressor => HOG
- Gradient Boosting Classifier => HINGE
- SVM => COLD

In [55]:
final_train= np.concatenate((svm_cold.predict(COLD_feature_training).reshape(-1, 1), gradient_booster_hinge.predict(HINGE_feature_training).reshape(-1, 1)), axis=1)
final_train = np.concatenate((final_train, lg_hog.predict(HOG_feature_training).reshape(-1, 1)), axis=1)

final_test = np.concatenate((svm_cold.predict(COLD_feature_testing).reshape(-1, 1), gradient_booster_hinge.predict(HINGE_feature_testing).reshape(-1, 1)), axis=1)
final_test = np.concatenate((final_test, lg_hog.predict(HOG_feature_testing).reshape(-1, 1)), axis=1)

In [60]:
final_test[:15]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 0],
       [1, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0]])

In [61]:
test_predicted[:15]

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0])

In [57]:
final_y = y_train
clf = LogisticRegression(random_state=42, max_iter=1000, C=10, class_weight="balanced")
clf = clf.fit(final_train, final_y)
test_predicted = clf.predict(final_test)
print(f"Testing accuracy is %0.2f"%(np.sum(test_predicted == y_test)/len(y_test) * 100))

Testing accuracy is 82.19


In [62]:
from sklearn.feature_selection import RFECV

In [ ]:
rfecv = RFECV(
    estimator=Lasso(),
    cv=3,
    scoring="r2",
    n_jobs=-1,
    min_features_to_select=2,
)
refcv.fit(X_train, y_train)

refcv.support_